In [362]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import math
import scipy.stats


In [363]:
data=pd.read_csv("iris.data")
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
data.describe()

,SepalLength,SepalWidth,PetalLength,PetalWidth
count,149.000000,149.000000,149.000000,149.000000
mean,5.848322,3.051007,3.774497,1.205369
std,0.828594,0.433499,1.759651,0.761292
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.400000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [364]:
# Get your data in matrix (in X, Y) form
X = np.zeros((149,4), dtype = float)
Y = np.zeros((149))

X = data.iloc[:, :4].values
Y = data.iloc[:,-1].values
le = LabelEncoder()
Y = le.fit_transform(Y)
#for i in range(149):
#    for j in range(4)

In [365]:
# TODO: You have to implement the following class
## Your code goes here...
class KNearestNeighbor:
    ''' Implements the KNearest Neigbours For Classification... '''
    def __init__(self, k, scalefeatures=False):
        self.k = k
        
    def Euclidian_Distance(self,Observe,Actual):
        distance=0
        for x in range(len(Observe)):
            distance += pow((Observe[x] - Actual[x]), 2)
        return math.sqrt(distance)
        
    
    def compute_distances_two_loops(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists1 = np.zeros((num_test, num_train))
        dists2 = np.zeros((num_test, num_train))
        for i in range(num_test):
            for j in range(num_train):
                dists2[i,j] = np.sqrt(np.sum((X[i] - self.X_train[j]) ** 2))
        #print(dist2)
        return dists2

    def compute_distances_one_loop(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        for i in range(num_test):
            dists[i] = np.sqrt(np.sum((self.X_train - X[i]) ** 2, axis=1))
        print(dists)
        return dists

    def compute_distances_no_loops(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train)) 
        val1 = np.sum(X ** 2, axis=1)
        val2 = np.sum(self.X_train ** 2, axis=1)
        res = val1.reshape((num_test, 1)) + val2 - 2 * X.dot(self.X_train.T)
        dists = np.sqrt(res)
        #print(dists)
        return (dists)
    
    def train(self, X, Y):
        nexamples,nfeatures=X.shape
        self.X_train = X
        self.Y_train = Y        
    
    def predict(self, X, methodtype='noloops'):
        num_test = X.shape[0]
        
        y_pred = np.zeros(num_test, dtype = self.Y_train.dtype)
        pclass=[]
        
        # defining a function variable so that you will only need to call compute_distance...
        if methodtype == 'noloops':
            compute_distance = self.compute_distances_no_loops(X)
        elif methodtype == 'oneloop':
            compute_distance = self.compute_distances_one_loop(X)
        else:
            compute_distance = self.compute_distances_two_loops(X)
        num_test = compute_distance.shape[0]
        for i in range(num_test):
            closest_y = []
            knn_ix = compute_distance[i].argsort()[:3]
            closest_y = self.Y_train[knn_ix]
            values, counts = np.unique(closest_y, return_counts=True)
            y_pred[i] = values[counts == counts.max()].min()
        return y_pred
        

In [366]:
# Spllit Data into train and test
import Split as t
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)
print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape)   
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape)

 Training Data Set Dimensions= (104, 4) Training True Class labels dimensions (104,)
 Test Data Set Dimensions= (45, 4) Test True Class labels dimensions (104,)


In [367]:
# train a 3-nearest neighbour classifier...
KNN = KNearestNeighbor(3)
KNN.train(Xtrain, Ytrain)

In [370]:
# print predicted classes
test3 = KNN.predict(Xtest, "noloops")
print(test3)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2]


In [369]:
#Lets see how good we are doing, by finding the accuracy on the test set..
print("No Loops Accuracy Score: ",accuracy_score(test3,Ytest))

No Loops Accuracy Score:  0.9555555555555556
